In [1]:
!pip install kafka-python
!pip install tweepy

from kafka import KafkaConsumer
from kafka import KafkaClient

import numpy as np
import tweepy
import pandas as pd
import matplotlib.pyplot as plt

import json

In [2]:
# Make the graphs prettier
pd.set_option('display.mpl_style', 'default')

/usr/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2882: FutureWarning: 
mpl_style had been deprecated and will be removed in a future version.
Use `matplotlib.pyplot.style.use` instead.

  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
def run():
        results = []
        consumer = KafkaConsumer(bootstrap_servers='localhost:9092',
                                 auto_offset_reset='earliest',
                                 consumer_timeout_ms=1000)
        consumer.subscribe(['Twitter'])

        for message in consumer:
            if(len(message.value)>50):
                j = json.loads(message.value)
                results.append(j)
        print("Number of tweets collected: "+str(len(results)))
        consumer.close()
        return results

In [29]:
results = []
results = run()
print(results[0])

Number of tweets collected: 260
{u'user': {u'screen_name': u'Kdthepuppy', u'id': u'911433459876298752', u'profile_image_url': u'http://pbs.twimg.com/profile_images/917301321035800577/8sivEMI2_normal.jpg', u'name': u'Kd'}, u'text': u'Check out this video of the warriors shaving points from tonight\u2019s game and see how they get away with it\u2026 https://t.co/CBKAyyUbOf', u'created_at': u'Wed Dec 20 21:05:46 PST 2017', u'location': {u'lat': u'0', u'lon': u'0'}, u'id': u'943709054781702144'}


In [30]:
def extract_hash_tags(s):
    relevantHashtags = ['#NFL', '#NHL', '#MLB', '#NBA'] 
    allHashtags = set(part[0:] for part in s.split() if part.startswith('#'))
    ans = list(set(allHashtags) & set(relevantHashtags))
    ans.sort()
    return ' '.join(ans)

In [31]:
# Create a function to convert a given list of tweets into a Pandas DataFrame.
# The DataFrame will consist of only the values, which I think might be useful for analysis...


def toDataFrame(tweets):

    DataSet = pd.DataFrame()

    DataSet['tweetID'] = [tweet["id"] for tweet in tweets]
    DataSet['tweetText'] = [tweet["text"] for tweet in tweets]
    DataSet['hashtags'] = [ extract_hash_tags(tweet["text"]) for tweet in tweets]
    DataSet['tweetCreated'] = [tweet["created_at"] for tweet in tweets]
    
    DataSet['userLocationLat'] = [tweet["location"]["lat"] for tweet in tweets]
    DataSet['userLocationLon'] = [tweet["location"]["lon"] for tweet in tweets]
    
    DataSet['userID'] = [tweet["user"]["id"] for tweet in tweets]
    DataSet['userScreen'] = [tweet["user"]["screen_name"] for tweet in tweets]
    DataSet['userName'] = [tweet["user"]["name"] for tweet in tweets]
    #DataSet['userTimezone'] = [tweet["user"]["time_zone"] for tweet in tweets]

    return DataSet

In [32]:
#Pass the tweets list to the above function to create a DataFrame
DataSet = toDataFrame(results)

In [33]:
DataSet.head(6)

,tweetID,tweetText,hashtags,tweetCreated,userLocationLat,userLocationLon,userID,userScreen,userName
0,943709054781702144,Check out this video of the warriors shaving p...,,Wed Dec 20 21:05:46 PST 2017,0,0,911433459876298752,Kdthepuppy,Kd
1,943709121253183488,#NFL Check out Zippo Lighter - NFL Throwback C...,#NFL,Wed Dec 20 21:06:02 PST 2017,0,0,855206706296627201,TrendWerks,TrendWerks
2,943709122582675456,#NFL Check out Zippo Lighter - NFL Throwback C...,#NFL,Wed Dec 20 21:06:02 PST 2017,0,0,87231951,Millsports,TRENDJUNKIE
3,943709111383941120,RT @ProCityHoops: Murder reported in Oklahoma ...,#NBA,Wed Dec 20 21:06:00 PST 2017,0,0,134385491,UoenoJACK,J.Jackson
4,943709129939542016,"RT @TamaraJolee: Dirk brings kids to tears, J....",,Wed Dec 20 21:06:04 PST 2017,0,0,97054286,AudraOlsen,Audra Olsen
5,943709137124204544,RT @kishownstarmaps: 濃い上に酔っぱらいだからとっ散らかっちゃって… h...,,Wed Dec 20 21:06:06 PST 2017,0,0,3142999214,38726936e1264db,ゆうり@文ストは神d('∀'*)


In [34]:
DataSet.shape

(260, 9)

In [35]:
# 'None' is treated as null here, so I'll remove all the records having 'None' in their 'hashtags' column
DataSet = DataSet[DataSet.hashtags.notnull()]
# Let's also check how many records are we left with now
len(DataSet)

260

In [36]:
hashtagCount = DataSet['hashtags'].value_counts()[:6]
print hashtagCount

        112
#NBA     64
#NFL     45
#MLB     34
#NHL      5
Name: hashtags, dtype: int64


In [37]:
# Create a bar-graph figure of the specified size
plt.rcParams['figure.figsize'] = (15, 5)

# Plot the Time Zone data as a bar-graph
hashtagCount[1:].plot(kind='bar')


# Assign labels and title to the graph to make it more presentable
plt.xlabel('Hashtags #')
plt.ylabel('Tweet Count')
plt.title('Count of tweets with the hastags specified')
plt.savefig('TweetsVsCounts.png')